In [1]:
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess

from torch import nn
import pandas as pd
import os

from tqdm.auto import tqdm, trange

from sklearn.metrics import classification_report
import json

from torch.utils.data import Dataset
import torch.utils.data as data
from torch import optim
from torch.autograd import Variable
import numpy as np
from sentence_transformers import util
from nltk.tokenize import TweetTokenizer
import re, string

/homes/yg007/nytimes_project/venv_nyc/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from lavis.models import model_zoo

In [3]:
# print(model_zoo)

In [4]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"

# Load the model

In [5]:
# setup device to use
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:
model, vis_processors, txt_processors = load_model_and_preprocess(
    name = "clip_feature_extractor", model_type="ViT-B-32", is_eval=True, device=device
)

In [7]:
tt = TweetTokenizer()

# Define the Dataset class

In [8]:
class TwitterCOMMsDataset(Dataset):
    def __init__(self, csv_path, img_dir):
        """
        Args:
            csv_path (string): Path to the {train_completed|val_completed}.csv file.
            image_folder_dir (string): Directory containing the images
        """
        self.df = pd.read_csv(csv_path, index_col=0)
        self.img_dir = img_dir
        
        self.df['exists'] = self.df['filename'].apply(lambda filename: os.path.exists(os.path.join(img_dir, filename)))
        delete_row = self.df[self.df["exists"]==False].index
        self.df = self.df.drop(delete_row)
    
    def __len__(self):
        return len(self.df)
    
    def remove_URL(self, text):
        """Remove URLs from a sample string"""
        return re.sub(r"http\S+", '', text)
    
    def remove_punc(self, text):
        """Remove punctuation from a sample string"""
        return re.sub(r'[^\w\s]', '', text)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        
        topic = item['topic']
        falsified = int(item['falsified'])
        not_falsified = float(not item['falsified'])
#         label = np.array((falsified, not_falsified))
        label = np.array(falsified)
        domain = topic.split('_')[0]
        diff = topic.split('_')[1]
        
        # Preprocess the text
        caption = item['full_text']
        caption = ' '.join(tt.tokenize(caption))
        caption = self.remove_punc(self.remove_URL(caption))
        
        # Open the image
        img_filename = item['filename']
        raw_image = Image.open(os.path.join(self.img_dir, img_filename)).convert('RGB')
        
        # Get the multimodal embedding
        image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
        text_input = txt_processors["eval"](caption)
        sample = {"image": image, "text_input": [text_input]}   # image shape: [1, 3, 224, 224]
        features = model.extract_features(sample)
        features_image = features.image_embeds   # [1, 512]
        features_text = features.text_embeds   # [1, 512]
#         multimodal_emb = torch.cat((features_image, features_text), 1)
        multimodal_emb = features_image * features_text
        
        cos_sim = util.cos_sim(features_text, features_image)
#         features_image_proj = features_image.image_embeds_proj[:,0,:]   # [1, 256]
#         features_text_proj = features_text.text_embeds_proj[:,0,:]   # [1, 256]
        
#         multimodal_emb = torch.cat((features_image_proj, features_text_proj), 1)
#         multimodal_emb = features_image_proj * features_text_proj   # [1, 256]
#         print(multimodal_emb.shape)

#         similarity = features_image_proj @ features_text_proj.t()

        return {"multimodal_emb": multimodal_emb,
                "topic": topic, 
                "label": label, 
                "domain": domain, 
                "difficulty": diff,
               "similarity": cos_sim}
        
        
        
    

In [9]:
# class ClimateAndCovidDataset(Dataset):
#     def __init__(self, csv_path, img_dir):
#         """
#         Args:
#             csv_path (string): Path to the {train_completed|val_completed}.csv file.
#             image_folder_dir (string): Directory containing the images
#         """
#         self.df = pd.read_csv(csv_path, index_col=0)
#         self.img_dir = img_dir
        
#         self.df['exists'] = self.df['filename'].apply(lambda filename: os.path.exists(os.path.join(img_dir, filename)))
#         delete_row = self.df[self.df["exists"]==False].index
#         self.df = self.df.drop(delete_row)
        
#         self.df['is_military'] = self.df['topic'].apply(lambda topic: 'military' in topic)
#         delete_row = self.df[self.df["is_military"]==True].index
#         self.df = self.df.drop(delete_row)
    
#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         item = self.df.iloc[idx]
#         caption = item['full_text']
#         img_filename = item['filename']
#         topic = item['topic']
#         falsified = float(item['falsified'])
#         not_falsified = float(not item['falsified'])
#         label = np.array((falsified, not_falsified))
#         domain = topic.split('_')[0]
#         diff = topic.split('_')[1]
        
#         try:
#             raw_image = Image.open(os.path.join(self.img_dir, img_filename)).convert('RGB')
#             image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
#             text_input = txt_processors["eval"](caption)
#             sample = {"image": image, "text_input": [text_input]}   # image shape: [1, 3, 224, 224]
        
#             features_multimodal = model.extract_features(sample, mode="multimodal")
# #             features_image = model.extract_features(sample, mode="image")
# #             features_text = model.extract_features(sample, mode="text")
# #             print(features_multimodal.multimodal_embeds[:, 0, :].shape)
        
#             return {"multimodal_emb": features_multimodal.multimodal_embeds[:, 0, :],
#                     "topic": topic, 
#                     "label": label, 
#                     "domain": domain, 
#                     "difficulty": diff}
        
#         except IOError as e:
#             print(e)
        
        
        
    

In [10]:
# train_data = TwitterCOMMsDataset(csv_path='../data/train_completed.csv',
#                                     img_dir='/import/network-temp/yimengg/data/twitter-comms/train/images/train_image_ids')   # took ~one hour to construct the dataset
val_data = TwitterCOMMsDataset(csv_path='../data/val_completed.csv', 
                               img_dir='/import/network-temp/yimengg/data/twitter-comms/images/val_images/val_tweet_image_ids')

In [11]:
val_data[0]['multimodal_emb'].shape

torch.Size([1, 512])

In [12]:
# train_data.__len__()

In [13]:
BATCH_SIZE = 32

# train_iterator = data.DataLoader(train_data, 
#                                  shuffle = True, 
#                                  batch_size=BATCH_SIZE)
val_iterator = data.DataLoader(val_data, 
                               shuffle = False, 
                               batch_size=BATCH_SIZE)

In [14]:
class Net(nn.Module):
    def __init__(self, in_dim, out_dim=2):
        super(Net, self).__init__()
        
        self.fc = nn.Linear(in_dim, out_dim)
        self.in_dim = in_dim
    
    def forward(self, x):
        x = x.view(-1, self.in_dim)
        out = self.fc(x)
        return out
    
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

In [15]:
def normal_init(m, mean, std):
    if isinstance(m, nn.Linear):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

In [20]:
def main():
    net = Net(512)
#     net = Net(1024)
    net.cuda()
    net.train()
    net.weight_init(mean=0, std=0.02)
    
    lr = 0.0001
    optimizer = optim.Adam(net.parameters(), lr=lr, betas=(0.5, 0.999), weight_decay=1e-5)
    
    criterion = nn.CrossEntropyLoss()
    criterion.to(device)
    
    softmax = nn.Softmax(dim=1)
 
    EPOCHS = 2
    for epoch in range(EPOCHS):
        total_loss = 0
        num_correct = 0
        total = 0
        for i, batch in tqdm(enumerate(val_iterator, 0), desc='iterations'):
            inputs = batch["multimodal_emb"].to(device)
            labels = batch["label"].to(device)
            inputs, labels = Variable(inputs), Variable(labels)
            
            net.zero_grad()
            y_preds = net(inputs)
            loss = criterion(y_preds, labels)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
#             _, top_pred = y_preds.topk(1, 1)
            
            top_pred = torch.zeros_like(labels)
            y_preds = softmax(y_preds)
#             print(y_preds[:, 0])
            top_pred[y_preds[:, 1] >= 0.5] = 1
            y = labels.cpu()
            batch_size = y.shape[0]
            top_pred = top_pred.cpu().view(batch_size)
            
#             num_correct += sum(top_pred == y[:, 0]).item()
            num_correct += sum(top_pred == y).item()
            total += batch_size
            
            if i % 50 == 0:
                print("Epoch [%d/%d]: Training accuracy %.2f, training loss %.3f" % (epoch+1, EPOCHS, num_correct/total, total_loss/total))
                print(y_preds[:, 1])
#                 print(top_pred)
#                 print(labels)
#                 print(num_correct)
#                 print(total)

    return net

In [21]:
net = main()

iterations: 1it [00:01,  1.65s/it]

Epoch [1/2]: Training accuracy 0.56, training loss 0.022
tensor([0.4662, 0.4662, 0.4780, 0.4651, 0.4848, 0.4562, 0.4837, 0.4837, 0.4893,
        0.4970, 0.4970, 0.4867, 0.4793, 0.4793, 0.4764, 0.4680, 0.4674, 0.4655,
        0.4655, 0.4979, 0.5008, 0.5283, 0.5283, 0.5076, 0.4800, 0.4800, 0.4818,
        0.5099, 0.5035, 0.4918, 0.4918, 0.4911], device='cuda:0',
       grad_fn=<SelectBackward0>)


iterations: 51it [01:49,  2.73s/it]

Epoch [1/2]: Training accuracy 0.53, training loss 0.022
tensor([0.4662, 0.4916, 0.4608, 0.5038, 0.5038, 0.5075, 0.5075, 0.4882, 0.5114,
        0.4845, 0.4845, 0.4939, 0.5137, 0.4949, 0.4949, 0.4915, 0.4845, 0.4845,
        0.4754, 0.4639, 0.4498, 0.4498, 0.4770, 0.4947, 0.4947, 0.4901, 0.4670,
        0.5153, 0.4863, 0.4863, 0.5044, 0.4761], device='cuda:0',
       grad_fn=<SelectBackward0>)


iterations: 101it [03:21,  1.89s/it]

...

iterations: 601it [19:07,  2.24s/it]

Epoch [2/2]: Training accuracy 0.70, training loss 0.019
tensor([0.2586, 0.5460, 0.2935, 0.3680, 0.3680, 0.5789, 0.4646, 0.4564, 0.3394,
        0.3702, 0.3702, 0.5956, 0.5500, 0.5500, 0.4736, 0.4809, 0.4809, 0.4583,
        0.4248, 0.4585, 0.3036, 0.3036, 0.4746, 0.3671, 0.3671, 0.3572, 0.5105,
        0.3937, 0.3937, 0.4639, 0.4014, 0.5345], device='cuda:0',
       grad_fn=<SelectBackward0>)


iterations: 651it [20:45,  1.90s/it]

Epoch [2/2]: Training accuracy 0.70, training loss 0.019
tensor([0.4919, 0.4919, 0.4307, 0.4311, 0.4311, 0.4667, 0.4553, 0.4553, 0.5320,
        0.4723, 0.3911, 0.3911, 0.5026, 0.4012, 0.4961, 0.3398, 0.3398, 0.4972,
        0.4972, 0.5515, 0.6199, 0.4279, 0.4279, 0.4519, 0.6008, 0.4721, 0.4071,
        0.4071, 0.5791, 0.5791, 0.5849, 0.5849], device='cuda:0',
       grad_fn=<SelectBackward0>)


iterations: 691it [21:54,  1.90s/it]


In [ ]:
!nvidia-smi

In [22]:
num_correct = 0
total = 0
for i, batch in tqdm(enumerate(val_iterator, 0), desc='iterations'):
    similarity = batch["similarity"].squeeze()
    labels = batch["label"]
    y_preds = torch.zeros_like(labels)
#     y_preds[similarity < 0.25] = 1
    y_preds[similarity < 0.5] = 1
    
    num_correct += sum(y_preds == labels).item()
    total += BATCH_SIZE
    
    if i % 50 == 0:
        print("Accuracy %.2f" % (num_correct/total))
        print(y_preds)
        print(similarity)

iterations: 1it [00:01,  1.54s/it]

Accuracy 0.50
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([0.3349, 0.3349, 0.2642, 0.1821, 0.2555, 0.2151, 0.3025, 0.3025, 0.2159,
        0.3478, 0.3478, 0.2324, 0.2302, 0.2302, 0.2318, 0.2411, 0.2512, 0.2805,
        0.2805, 0.1719, 0.2577, 0.2763, 0.2763, 0.2854, 0.3493, 0.3493, 0.2674,
        0.1792, 0.1598, 0.2364, 0.2364, 0.1451], device='cuda:0',
       grad_fn=<SqueezeBackward0>)


iterations: 51it [01:48,  2.79s/it]

Accuracy 0.49
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([0.2013, 0.2949, 0.2541, 0.2889, 0.2889, 0.3252, 0.3252, 0.1864, 0.2455,
        0.3206, 0.3206, 0.2248, 0.1014, 0.3531, 0.3531, 0.2052, 0.2658, 0.2658,
        0.2847, 0.2716, 0.2694, 0.2694, 0.1450, 0.2631, 0.2631, 0.1578, 0.2307,
        0.2952, 0.3938, 0.3938, 0.2566, 0.2436], device='cuda:0',
       grad_fn=<SqueezeBackward0>)


iterations: 57it [02:00,  2.12s/it]


KeyboardInterrupt: 

# Playground

In [ ]:
val_df = pd.read_csv('../data/val_completed.csv', index_col=0)
val_df

In [ ]:
img_dir = '/import/network-temp/yimengg/data/twitter-comms/images/val_images/val_tweet_image_ids'
val_df['exists'] = val_df['filename'].apply(lambda filename: os.path.exists(os.path.join(img_dir, filename)))

In [ ]:
delete_row = val_df[val_df["exists"]==False].index

In [ ]:
val_df = val_df.drop(delete_row)

In [ ]:
val_df

In [ ]:
def remove_URL(text):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", '', text)

def remove_punc(text):
    """Remove punctuation from a sample string"""
    return re.sub(r'[^\w\s]', '', text)

s = val_data.df.iloc[0]['full_text']
print(s)
s = ' '.join(tt.tokenize(s))
print(remove_punc(remove_URL(s)))

In [ ]:
idx = 6
print(val_data.df.iloc[idx]['full_text'])
print("Falsified? " + str(val_data.df.iloc[idx]['falsified']))
print(val_data.df.iloc[idx]['topic'])
print(val_data[idx]['similarity'])

raw_image = Image.open('/import/network-temp/yimengg/data/twitter-comms/images/val_images/val_tweet_image_ids/'+val_data.df.iloc[idx]['filename']).convert('RGB')   
display(raw_image.resize((596, 437)))